# Testing Gemma3 270m

In [ ]:
import os
import tensorflow as tf
import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2 as cv
import json
import kagglehub
import keras_hub

from time import sleep
from tqdm import trange
from scripts.dataset import Dataset

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sns.set_theme(palette=sns.color_palette("rocket_r"))

In [ ]:
assert os.path.isfile("kaggle.json"), "Kaggle credentials are required to run this project."

credentials = None
with open("kaggle.json") as kaggle_json:
    credentials = json.load(kaggle_json)

print(f"Using credentials = {credentials}")
os.environ['KAGGLE_USERNAME'] = credentials["username"]
os.environ['KAGGLE_KEY'] = credentials["key"]

In [ ]:
CURRENT_PATH = os.getcwd()

DATASET_PATH = os.path.join(CURRENT_PATH, "dataset.csv")
EXPERIMENT_PATH = os.path.join(CURRENT_PATH, '__experiments__')
EXPERIMENT_NAME = "llm_model"

# Trainig Parameters
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNEL = 3

# LLM Prediction (seconds)
DELAY_BEFORE_NEXT = 0.5
CHECKPOINT_PATH = os.path.join(CURRENT_PATH, "report-gemma3-270m.csv")

In [ ]:
assert os.path.isfile(DATASET_PATH), "Dataset not exists. Please run the previous notebook."
test_dataset = Dataset(dataset_path=DATASET_PATH, dataset_mode="csv", partition_name="test", preload=True, shuffle=False)

In [ ]:
def as_numpy(dataset: Dataset):
    n = len(dataset)
    iterator = iter(dataset)
    x = np.ndarray(shape=(n, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNEL), dtype=np.float64)
    y = np.ndarray(shape=(n,), dtype=np.int8)
    names = np.ndarray(shape=(n,), dtype=object)
    for index in trange(n):
        xi, yi, image_name = next(iterator)
        xi = cv.resize(xi, [IMAGE_WIDTH, IMAGE_HEIGHT])
        x[index] = xi
        y[index] = yi
        names[index] = image_name
    return x, y, names

x_test, y_test, names = as_numpy(test_dataset)
print(f"Image Shape = {x_test[0].shape}")

## Preparing the LLM

In [ ]:
path = kagglehub.model_download("keras/gemma3/keras/gemma3_270m")
print("Path to model files:", path)

In [ ]:
# model = keras_hub.models.Gemma3CausalLM.from_preset(path)
# model.generate("Keras is a", max_length=30)

In [ ]:
y_pred = np.ndarray(shape=y_test.shape, dtype=np.int8)

checkpoint = {
    "image_name": [],
    "y_true": [],
    "y_pred": [],
    "y_true_as_str": [],
    "y_pred_as_str": [],
}

# Iterate over the test dataset
for index in trange(len(x_test)):
    xi = x_test[index]
    yi = y_test[index]
    name = names[index]
    
    # Prediction...
    prediction = 0
    
    # Saving results
    y_pred[index] = prediction
    
    # Saving checkpoint
    checkpoint["image_name"].append(name)
    checkpoint["y_true"].append(yi)
    checkpoint["y_pred"].append(prediction)
    checkpoint["y_true_as_str"].append(test_dataset.decode_label(yi))
    checkpoint["y_pred_as_str"].append(test_dataset.decode_label(prediction))
    pd.DataFrame.from_dict(checkpoint).to_csv(CHECKPOINT_PATH, sep=";", index=False)

    # Delay before next prediction
    sleep(DELAY_BEFORE_NEXT)

## Evaluating the model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
from prettytable import PrettyTable

def plot_metrics(y_true: np.array, y_pred: np.array):
    table = PrettyTable(["Metric", "Value (%)"])
    
    # Calculating metrics...
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0.0) * 100
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0.0) * 100
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0.0) * 100

    table.add_row(['Accuracy', f"{accuracy:.2f}%"])
    table.add_row(['Precision', f"{precision:.2f}%"])
    table.add_row(['Recall', f"{recall:.2f}%"])
    table.add_row(['F1 Score', f"{f1:.2f}%"])
    
    print(table)

def plot_cm(y_true: np.array, y_pred: np.array):
    plt.figure(figsize=(8,6))
    cm = confusion_matrix(y_true, y_pred)
    plt.title(f'Confusion Matrix')
    sns.heatmap(cm, annot=True, cmap=sns.color_palette("rocket_r", as_cmap=True), fmt='.0f', yticklabels=test_dataset.classes, xticklabels=test_dataset.classes)
    plt.show()

In [ ]:
plot_metrics(y_test, y_pred)
plot_cm(y_test, y_pred)

Visualizing some errors

In [ ]:
errors = []
for index in range(len(y_pred)):
    true = y_test[index] 
    pred = y_pred[index]
    if true != pred:
        errors.append((x_test[index], true, pred))

In [ ]:
nrows = 5
ncols = 4

figure, axis = plt.subplots(nrows, ncols, squeeze=False, figsize=(14,18))
figure.suptitle(f"Errors")
axes = []
for ax in axis:
    axes += [*ax]
for index in range(nrows * ncols):
    ax = axes[index]
    x, true_class, pred = errors[index]
    pred = test_dataset.decode_label(pred)
    true_class = test_dataset.decode_label(true_class)
    ax.axis("off")
    ax.set_title(f"[y_true={true_class}, y_pred={pred}]")
    ax.imshow(x.astype(np.int64))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
plt.subplots_adjust(wspace=0, hspace=.2)
plt.show()